In [1]:
import argparse
import pickle
from pathlib import Path

import fraud_eagle as feagle
import fraudar
import networkx as nx
import numpy as np
import pandas as pd
import rsd

from rev2 import rev2compute
from utils import load_data, split_data_by_time

import seaborn as sns
import matplotlib.pyplot as plt

import sklearn.metrics

import warnings
from sklearn.exceptions import UndefinedMetricWarning
warnings.filterwarnings(action='ignore', category=UndefinedMetricWarning)

In [2]:
def get_results(data_name="alpha", alg="fraudar", attack_name="ilp_attack", budget=100, frac=0.2):
    data_nw_df, data_gt_df = load_data(data_name=data_name)
    p = Path(f"../res/{attack_name}/{alg}-{data_name}/{budget}-{frac}.pkl")
    if not p.exists():
        print(p)
        return None
    with open(p, "rb") as fp:
        scores = pickle.load(fp)
        scores = dict(zip(range(len(scores)), scores))
        # 1 means fraudster, 0 means benign
        scores["truth"] = dict(zip(data_gt_df["id"], (1-data_gt_df["label"])/2))
    # df = pd.DataFrame({sp: {u: scores[sp][u] for u in scores[sp] if u in data_gt_df["id"].tolist()} for sp in scores})
    df = pd.DataFrame({sp: {u: scores[sp][u] for u in scores[sp]} for sp in scores})
    df.loc[[u[:5]=="usock" for u in df.index.tolist()], "truth"] = 1
    df.loc[[u[:6]=="udummy" for u in df.index.tolist()], "truth"] = 1
    return df

In [3]:
def get_metrics(df_res, splits=4, K=None, th=0.5):
    if df_res is None:
        return None
    ret = {}
    for sp in range(splits):
        # df_res["truth"] = df_res["truth"].fillna(1)
        df_res.loc[[u[:5]=="usock" for u in df_res.index.tolist()], "truth"] = 1
        df_res.loc[[u[:6]=="udummy" for u in df_res.index.tolist()], "truth"] = 1
        df = df_res[[sp, "truth"]].dropna()
        df[sp]
        t = df["truth"].sum() if K is None else K
        t_frac = t / df.shape[0]
        df["pred"] = df[sp] >= th

        ret[sp] = {
            "precision": sklearn.metrics.precision_score(y_true=df["truth"], y_pred=df["pred"]),
            "recall": sklearn.metrics.recall_score(y_true=df["truth"], y_pred=df["pred"]),
            "f1": sklearn.metrics.f1_score(y_true=df["truth"], y_pred=df["pred"])
        }
    return ret

In [25]:
alg_list = ["fraudar", "rsd", "rev2", "sg", "fbox"][3:4]
data_list = ["alpha", "otc", "amazon", "epinions"][3:]
budget_list = [100.0, 200.0, 300.0, 400.0]
frac_list = [0.0, 0.2, 0.4, 0.6, 0.8]

In [26]:
attack_res = {
    (d, a, b, f): get_results(data_name=d, alg=a, attack_name="ilp_attack", budget=b, frac=f)
    for d in data_list for a in alg_list
    for b in budget_list for f in frac_list
}

In [27]:
# attack_res[("epinions", "fraudar", 400.0, 0.2)].to_csv("ar.csv")

In [28]:
th_dict = {
    "fraudar": 0.5,
    "rsd": 1.2,
    "rev2": 0.1,
    "fbox": 0.5,
    "sg": 0.5,
}
dict_met = {
    (d, a, b, f): pd.DataFrame.from_dict(get_metrics(df_res=attack_res[(d, a, b, f)], splits=4, K=None, th=th_dict[a]))
    for d, a, b, f in attack_res
}
df_met = pd.concat(dict_met, keys=dict_met.keys())
display(df_met)

0         1         2         3
epinions sg 100.0 0.0 precision  0.003340  0.002874  0.004188  0.003386
                      recall     0.011345  0.010309  0.014760  0.012821
                      f1         0.005160  0.004494  0.006525  0.005357
                  0.2 precision  0.003340  0.002874  0.004188  0.003386
                      recall     0.011129  0.009901  0.014134  0.012195
                      f1         0.005138  0.004454  0.006462  0.005300
                  0.4 precision  0.003340  0.002874  0.004188  0.003386
                      recall     0.010920  0.009524  0.013559  0.011628
                      f1         0.005115  0.004415  0.006400  0.005245
                  0.6 precision  0.003340  0.002874  0.004188  0.003386
                      recall     0.010720  0.009174  0.013029  0.011111
                      f1         0.005093  0.004376  0.006339  0.005190
                  0.8 precision  0.003340  0.002874  0.004188  0.003386
                      recall     0.010526  0.008850  0.012539  0.010638
                      f1         0.005071  0.004338  0.006279  0.005137
            200.0 0.0 precision  0.003340  0.002874  0.004188  0.003386
                      recall     0.011345  0.010309  0.014760  0.012821
                      f1         0.005160  0.004494  0.006525  0.005357
                  0.2 precision  0.003340  0.002874  0.004188  0.003386
                      recall     0.010920  0.009524  0.013559  0.011628
                      f1         0.005115  0.004415  0.006400  0.005245
                  0.4 precision  0.003340  0.002874  0.004188  0.003386
                      recall     0.010526  0.008850  0.012539  0.010638
                      f1         0.005071  0.004338  0.006279  0.005137
                  0.6 precision  0.003340  0.002874  0.004188  0.003386
                      recall     0.010160  0.008264  0.011662  0.009804
                      f1         0.005027  0.004264  0.006163  0.005034
                  0.8 precision  0.003340  0.002874  0.004188  0.003386
                      recall     0.009818  0.007752  0.010899  0.009091
                      f1         0.004984  0.004193  0.006051  0.004934
            300.0 0.0 precision  0.003340  0.002874  0.004188  0.003386
                      recall     0.011345  0.010309  0.014760  0.012821
                      f1         0.005160  0.004494  0.006525  0.005357
                  0.2 precision  0.003340  0.002874  0.004188  0.003386
                      recall     0.010720  0.009174  0.013029  0.011111
                      f1         0.005093  0.004376  0.006339  0.005190
                  0.4 precision  0.003340  0.002874  0.004188  0.003386
                      recall     0.010160  0.008264  0.011662  0.009804
                      f1         0.005027  0.004264  0.006163  0.005034
                  0.6 precision  0.003340  0.002874  0.004188  0.003386
                      recall     0.009655  0.007519  0.010554  0.008772
                      f1         0.004963  0.004158  0.005997  0.004886
                  0.8 precision  0.003340  0.002874  0.004188  0.003386
                      recall     0.009198  0.006897  0.009639  0.007937
                      f1         0.004900  0.004057  0.005839  0.004747
            400.0 0.0 precision  0.003340  0.002874  0.004188  0.003386
                      recall     0.011345  0.010309  0.014760  0.012821
                      f1         0.005160  0.004494  0.006525  0.005357
                  0.2 precision  0.003340  0.002874  0.004188  0.003386
                      recall     0.010526  0.008850  0.012539  0.010638
                      f1         0.005071  0.004338  0.006279  0.005137
                  0.4 precision  0.003340  0.002874  0.004188  0.003386
                      recall     0.009818  0.007752  0.010899  0.009091
                      f1         0.004984  0.004193  0.006051  0.004934
                  0.6 precision  0.003340  0.002874  0.004188  0.003386
        

In [29]:
def get_alg_data_met(df_met, alg="rev2", data="alpha", met="f1"):
    df_met.index = df_met.index.set_names(["data", "algo", "budget", "frac", "metrics"])
    rdf = df_met.mean(axis=1).to_frame().reset_index().pivot(index=["data", "algo", "frac", "metrics"], columns=["budget"]).loc[(data, alg, slice(None), met)]#.droplevel(0, axis=1).droplevel([0, 1, 3], axis=0)
    rdf.index = [f"baseline-{i}" if i > 0 else "no-attack" for i in rdf.index]
    return rdf

In [33]:
get_alg_data_met(df_met, alg="sg", data="epinions", met="precision")

0                              
budget           100.0     200.0     300.0     400.0
no-attack     0.003447  0.003447  0.003447  0.003447
baseline-0.2  0.003447  0.003447  0.003447  0.003447
baseline-0.4  0.003447  0.003447  0.003447  0.003447
baseline-0.6  0.003447  0.003447  0.003447  0.003447
baseline-0.8  0.003447  0.003447  0.003447  0.003447